In [1]:
import numpy as np
import tensorflow as tf

In [3]:
opt = tf.keras.optimizers.RMSprop(learning_rate=0.1)
var1 = tf.Variable(10.0)
loss = lambda: (var1 ** 2) / 2.0    # d(loss) / d(var1) = var1
step_count = opt.minimize(loss, [var1]).numpy()
var1.numpy()

9.683772

In [4]:
var1

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=9.683772>